# Limpieza datos combinados Usuarios y Ordenes de trabajo Zentry - *Piloto Pacífico*

Se genera combinación de datos propios de los usuario, con ordenes de trabajo (odt) para años 2022-2024 y se cuenta con anomalías confirmadas hasta 2023.
Estas últimas permitirán marcar el caso positivo de anomalías y las demás ordenes serán nuestro negativo de anomalía. Tomando el caso como una regresión logistica multivariado.

Dicho lo anterior, debido al corte de fecha de las anomalías confirmadas se tendrá como muestra total las odt de 2022 y 2023 - Zona Pacífico.

In [2]:
import os

import numpy as np
import pandas as pd

from cedenar_anomalies.utils.paths import (
    data_processed_dir,
    data_raw_dir,
)

In [3]:
os.getcwd()

'/Users/frandak2/Documents/repos/cedenar_anomalies/notebooks'

In [4]:
pd.set_option("display.max_rows", None)  # Mostrar todas las filas
pd.set_option("display.max_columns", None)  # Mostrar todas las columnas

## Datos por año

#### 2023

In [10]:
# ruta_archivo = "J:\\JESSICA\\Jessica\\Rafael\\igrid\\cedenar\\python_zentr\\dataset_decantado_2023.csv"
ruta_archivo = data_processed_dir("dataset_decantado_2023.csv")
df_2023 = pd.read_csv(ruta_archivo)
decantado_paci = df_2023[df_2023.ZONA == 'PACIFICO']

/var/folders/pq/cm_j30pd6d5gvwryc4gttm2h0000gn/T/ipykernel_7698/3044468155.py:3: DtypeWarning: Columns (54) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2023 = pd.read_csv(ruta_archivo)


In [11]:
decantado_paci.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13502 entries, 5 to 66367
Data columns (total 57 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   item_288        13502 non-null  int64  
 1   PLAN_COMERCIAL  13502 non-null  object 
 2   NIVEL           13502 non-null  float64
 3   LATI_USU        13502 non-null  float64
 4   LONG_USU        13502 non-null  float64
 5   AREA            13502 non-null  object 
 6   ZONA            13502 non-null  object 
 7   Descripcion     2400 non-null   object 
 8   odt             13502 non-null  int64  
 9   orden           13502 non-null  int64  
 10  item_1442       13431 non-null  float64
 11  item_8          13031 non-null  object 
 12  item_237        11769 non-null  object 
 13  item_23         10844 non-null  object 
 14  item_24         10840 non-null  object 
 15  item_33         9604 non-null   float64
 16  item_598        9638 non-null   object 
 17  item_601        10667 non-null  obje

In [33]:
filas, columnas = df_2023.shape
filas

19425

## Duplicados

In [151]:
def revisar_y_guardar_duplicados(df, nombre_df):
    """
    Revisa si hay duplicados en las columnas 'orden' y 'odt' de un DataFrame y guarda los duplicados en un archivo CSV.

    Parámetros:
    - df (DataFrame): El DataFrame a revisar.
    - nombre_df (str): El nombre del DataFrame para usar en el nombre del archivo.
    """

    # Identificar duplicados basados en 'orden' y 'odt'
    duplicados = df.duplicated(subset=["orden", "odt"], keep=False)

    # Contar el número de duplicados
    num_duplicados = duplicados.sum()

    # Guardar los duplicados en un archivo CSV si existen
    if num_duplicados > 0:
        duplicados_df = df[duplicados]
        nombre_archivo = f"{nombre_df}_duplicados.csv"
        duplicados_df.to_csv(nombre_archivo, index=False)
        print(
            f"Se encontraron {num_duplicados} registros duplicados en {nombre_df} y se guardaron en {nombre_archivo}."
        )
    else:
        print(f"No se encontraron registros duplicados en {nombre_df}.")

In [460]:
# Revisar y guardar duplicados en df_2023
revisar_y_guardar_duplicados(df_2023, "df_2023")

Se encontraron 16374 registros duplicados en df_2022 y se guardaron en df_2022_duplicados.csv.
Se encontraron 8254 registros duplicados en df_2023 y se guardaron en df_2023_duplicados.csv.


In [462]:
filas, columnas = df_2023.shape
print(f"Registros 2023 - duplicados = {filas - 8254}")

Registros 2022 - duplicados = 17087
Registros 2023 - duplicados = 11171


### Eliminar duplicados

In [163]:
def eliminar_duplicados(df, nombre_df):
    """
    Elimina registros duplicados en las columnas 'orden' y 'odt' de un DataFrame, conservando solo la primera aparición.

    Parámetros:
    - df (DataFrame): El DataFrame a procesar.
    - nombre_df (str): El nombre del DataFrame para imprimir en los resultados.
    """

    # Eliminar duplicados, conservando la primera aparición
    df_sin_duplicados = df.drop_duplicates(subset=["orden", "odt"], keep="first")

    # Imprimir información sobre el DataFrame resultante
    print(
        f"Registros duplicados eliminados en {nombre_df}. Tamaño original: {len(df)}, Tamaño después de eliminar duplicados: {len(df_sin_duplicados)}"
    )

    return df_sin_duplicados

In [464]:
# Eliminar duplicados en df_2023
df_2023 = eliminar_duplicados(df_2023, "df_2023")

df_2023.to_csv("df_2023_sin_duplicados.csv", index=False)

print("DataFrames sin duplicados guardados en archivos CSV.")

Registros duplicados eliminados en df_2022. Tamaño original: 33461, Tamaño después de eliminar duplicados: 21257
Registros duplicados eliminados en df_2023. Tamaño original: 19425, Tamaño después de eliminar duplicados: 13380
DataFrames sin duplicados guardados en archivos CSV.


## Variables a prescindir

### Ubicación

In [466]:
# Guardar el nuevo dataframe en un archivo csv
df_2022.to_csv("df_2022_sin_nulos.csv", index=False)

### Inspección Estado Del Predio

In [468]:
print(df_2023["item_237"].unique())

[True nan]
[True nan]


In [470]:
df_2023["item_237"].unique()

array([True, nan], dtype=object)

In [472]:
# Reemplazar NaN con False en df_2023['item_237']
df_2023["item_237"] = df_2023["item_237"].replace(np.nan, False)

C:\Users\User\AppData\Local\Temp\ipykernel_13120\1591275600.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_2022['item_237'] = df_2022['item_237'].replace(np.nan, False)
C:\Users\User\AppData\Local\Temp\ipykernel_13120\1591275600.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_2023['item_237'] = df_2023['item_237'].replace(np.nan, False)


In [474]:
print(df_2023["item_237"].value_counts())

item_237
True     18786
False     2468
Name: count, dtype: int64
item_237
True     11706
False     1674
Name: count, dtype: int64


### Medida Activa

In [183]:
print(df_2023["item_23"].unique())

['SI' nan 'TRUE']
['SI' nan 'TRUE']


In [478]:
df_2023["item_23"] = df_2023["item_23"].replace({np.nan: "NO", "TRUE": "SI"})

In [480]:
print(df_2023["item_23"].value_counts())

item_23
SI    18289
NO     2965
Name: count, dtype: int64
item_23
SI    10760
NO     2620
Name: count, dtype: int64


### Medida Reactiva

In [191]:
print(df_2023["item_24"].unique())

['SI' nan 'NO' 'TRUE']
['NO' nan 'SI' 'TRUE']


In [484]:
df_2023["item_24"] = df_2023["item_24"].replace({np.nan: "NO", "TRUE": "SI"})

In [486]:
print(df_2023["item_24"].value_counts())

item_24
NO    14339
SI     6915
Name: count, dtype: int64
item_24
NO    8394
SI    4986
Name: count, dtype: int64


### Tipo Sello 1 Tapa Bornera

In [488]:
print(df_2023["item_601"].unique())

['DA' 'RA' nan 'GR' 'DT' 'FA' 'TR' 'RN' 'AO' 'DY' 'NE' 'AZ' 'VE' 'BT' 'CF'
 'BL' 'TA' 'AL' 'VR' 'AT' 'SH' 'AM' 'SA' 'RO']
['DA' nan 'DT' 'RA' 'TR' 'FA' 'DY' 'AZ' 'AO' 'CF' 'VT' 'VR' 'VE' 'NE' 'AT'
 'BT' 'RN' 'EP' 'GR' 'BL' 'TA' 'AL']


In [490]:
df_2023["item_601"] = df_2023["item_601"].replace(np.nan, "NOAPL")

In [492]:
conteo_nan = df_2023["item_601"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 0
Número de valores NaN: 0


### Sello 1 Tapa Principal

In [494]:
conteo_nan = df_2023["item_33"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 5879
Número de valores NaN: 3853


In [496]:
df_2023["item_33"] = df_2023["item_33"].replace(np.nan, "NO INDICA")

In [498]:
conteo_nan = df_2023["item_33"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 0
Número de valores NaN: 0


### Tipo Sello 1 Tapa Principal

In [500]:
conteo_nan = df_2023["item_598"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 5940
Número de valores NaN: 3821


In [502]:
df_2023["item_598"] = df_2023["item_598"].replace(np.nan, "NOAPL")

In [504]:
conteo_nan = df_2023["item_598"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 0
Número de valores NaN: 0


### Sello 2 Tapa Principal

In [506]:
conteo_nan = df_2023["item_35"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 8172
Número de valores NaN: 5215


In [508]:
df_2023["item_35"] = df_2023["item_35"].replace(np.nan, "NO INDICA")

In [510]:
conteo_nan = df_2023["item_35"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 0
Número de valores NaN: 0


### Tipo Sello 2 Tapa Principal

In [512]:
conteo_nan = df_2023["item_599"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 8257
Número de valores NaN: 5227


In [514]:
df_2023["item_599"] = df_2023["item_599"].replace(np.nan, "NOAPL")

In [516]:
conteo_nan = df_2023["item_599"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 0
Número de valores NaN: 0


### Tipo Sello Caja

In [518]:
conteo_nan = df_2023["item_603"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 8920
Número de valores NaN: 5598


In [520]:
df_2023["item_603"] = df_2023["item_603"].replace(np.nan, "NOAPL")

In [522]:
conteo_nan = df_2023["item_603"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 0
Número de valores NaN: 0


### Sello Caja

In [524]:
conteo_nan = df_2023["item_43"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 8944
Número de valores NaN: 5640


In [526]:
df_2023["item_43"] = df_2023["item_43"].replace(np.nan, "NO INDICA")

In [528]:
conteo_nan = df_2023["item_43"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 0
Número de valores NaN: 0


### Corriente (A)

In [530]:
conteo_nan = df_2023["item_108"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 9142
Número de valores NaN: 5431


In [532]:
df_2023["item_108"] = df_2023["item_108"].replace(np.nan, "NO INDICA")

In [534]:
conteo_nan = df_2023["item_108"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 0
Número de valores NaN: 0


### Tipo Sello 2 Tapa Bornera

In [536]:
conteo_nan = df_2023["item_602"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 14916
Número de valores NaN: 8816


In [538]:
df_2023["item_602"] = df_2023["item_602"].replace(np.nan, "NOAPL")

In [540]:
conteo_nan = df_2023["item_602"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 0
Número de valores NaN: 0


### Tipo Sello Home Display

In [542]:
conteo_nan = df_2023["item_597"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 17411
Número de valores NaN: 10931


In [544]:
df_2023["item_597"] = df_2023["item_597"].replace(np.nan, "NOAPL")

In [546]:
conteo_nan = df_2023["item_597"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 0
Número de valores NaN: 0


### Sin Sello O Sello Abierto En Caja

array([nan, True], dtype=object)

In [552]:
print(df_2023["item_74"].unique())

[nan True]
[nan True]


In [554]:
# Reemplazar NaN con False en df_2023['item_237']
df_2023["item_74"] = df_2023["item_74"].replace(np.nan, False)

C:\Users\User\AppData\Local\Temp\ipykernel_13120\581404203.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_2022['item_74'] = df_2022['item_74'].replace(np.nan, False)
C:\Users\User\AppData\Local\Temp\ipykernel_13120\581404203.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_2023['item_74'] = df_2023['item_74'].replace(np.nan, False)


In [556]:
print(df_2023["item_74"].value_counts())

item_74
False    18535
True      2719
Name: count, dtype: int64
item_74
False    12013
True      1367
Name: count, dtype: int64


### Tipo Acometida

array([nan, 'INSTALACION ACOM MONOFASICA', 'INSTALACION ACOM BIFASICA',
       'INSTALACION ACOM TRIFASICA'], dtype=object)

In [560]:
print(df_2023["item_248"].value_counts())

item_248
INSTALACION ACOM MONOFASICA    1341
INSTALACION ACOM BIFASICA       107
INSTALACION ACOM TRIFASICA       42
Name: count, dtype: int64
item_248
INSTALACION ACOM MONOFASICA    446
INSTALACION ACOM BIFASICA       47
INSTALACION ACOM TRIFASICA      19
Name: count, dtype: int64


In [562]:
conteo_nan = df_2023["item_248"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 19764
Número de valores NaN: 12868


In [564]:
df_2023["item_248"] = df_2023["item_248"].replace(np.nan, "NO INDICA")

In [566]:
conteo_nan = df_2023["item_248"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 0
Número de valores NaN: 0


### Sin Sello (S) Tapa Principal

array([nan, True], dtype=object)

In [571]:
conteo_nan = df_2023["item_68"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 21115
Número de valores NaN: 13366


In [573]:
df_2023["item_68"] = df_2023["item_68"].replace(np.nan, False)

C:\Users\User\AppData\Local\Temp\ipykernel_13120\3129199292.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_2022['item_68'] = df_2022['item_68'].replace(np.nan, False)
C:\Users\User\AppData\Local\Temp\ipykernel_13120\3129199292.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_2023['item_68'] = df_2023['item_68'].replace(np.nan, False)


In [575]:
conteo_nan = df_2023["item_68"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 0
Número de valores NaN: 0


In [577]:
df_2022["item_68"].unique()

array([False,  True])

### Descripcion

*Anomalias confirmadas*

In [580]:
df_2023["Descripcion"].unique()

array([nan, 'Anomalia verificada y aprobada',
       'Anomalia verificada con acta digital', 'Posible anomalia'],
      dtype=object)

In [582]:
df_2023["Descripcion"].unique()

array([nan, 'Anomalia verificada y aprobada',
       'Anomalia verificada con acta digital', 'Posible anomalia'],
      dtype=object)

In [584]:
print(df_2023["Descripcion"].value_counts())

Descripcion
Posible anomalia                        47
Anomalia verificada y aprobada          13
Anomalia verificada con acta digital     6
Name: count, dtype: int64
Descripcion
Anomalia verificada y aprobada          1612
Anomalia verificada con acta digital     676
Posible anomalia                         105
Name: count, dtype: int64


In [586]:
df_2023["Descripcion"] = df_2023["Descripcion"].replace(np.nan, "Solo odt")

In [588]:
print(df_2023["Descripcion"].value_counts())

Descripcion
Solo odt                                21188
Posible anomalia                           47
Anomalia verificada y aprobada             13
Anomalia verificada con acta digital        6
Name: count, dtype: int64
Descripcion
Solo odt                                10987
Anomalia verificada y aprobada           1612
Anomalia verificada con acta digital      676
Posible anomalia                          105
Name: count, dtype: int64


## Dataframes limpios

In [590]:
# Lista de columnas a conservar
columnas_a_conservar = [
    "item_68",
    "item_248",
    "item_74",
    "item_597",
    "item_602",
    "item_108",
    "item_43",
    "item_603",
    "item_599",
    "item_35",
    "item_598",
    "item_33",
    "item_601",
    "item_24",
    "item_23",
    "item_237",
    "LATI_USU",
    "LONG_USU",
    "NIVEL",
    "AREA",
    "item_288",
    "odt",
    "orden",
    "PLAN_COMERCIAL",
    "Descripcion",
]

df_2023_filtrado = df_2023[columnas_a_conservar]

# Guardar los DataFrames filtrados en archivos CSV
df_2023_filtrado.to_csv("df_2023_filtrado.csv", index=False)

print("\nDataFrames filtrados guardados en archivos CSV.")


DataFrames filtrados guardados en archivos CSV.


In [594]:
print("df_2023_filtrado:")
df_2023_filtrado.info()

df_2023_filtrado:
<class 'pandas.core.frame.DataFrame'>
Index: 13380 entries, 0 to 19424
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   item_68         13380 non-null  bool   
 1   item_248        13380 non-null  object 
 2   item_74         13380 non-null  bool   
 3   item_597        13380 non-null  object 
 4   item_602        13380 non-null  object 
 5   item_108        13380 non-null  object 
 6   item_43         13380 non-null  object 
 7   item_603        13380 non-null  object 
 8   item_599        13380 non-null  object 
 9   item_35         13380 non-null  object 
 10  item_598        13380 non-null  object 
 11  item_33         13380 non-null  object 
 12  item_601        13380 non-null  object 
 13  item_24         13380 non-null  object 
 14  item_23         13380 non-null  object 
 15  item_237        13380 non-null  bool   
 16  LATI_USU        13380 non-null  float64
 17  LONG_USU        13

## Marcar reincidencias

In [419]:
def crear_campo_reincidente(df):
    """
    Crea el campo 'reincidente' en un DataFrame basado en la columna 'item_288'.

    Parámetros:
    - df (DataFrame): El DataFrame a procesar.
    """

    # Identificar registros duplicados en 'item_288'
    duplicados = df.duplicated(subset="item_288", keep=False)

    # Crear el campo 'reincidente'
    df["reincidente"] = duplicados.apply(lambda x: "SI" if x else "NO")

    return df

In [596]:
df_2023_filtrado = crear_campo_reincidente(df_2023_filtrado.copy())

df_2023_filtrado.to_csv("df_2023_filtrado_reincidente.csv", index=False)

print("\nDataFrames modificados guardados en archivos CSV.")


DataFrames modificados guardados en archivos CSV.


In [600]:
print("df_2023_filtrado:")
df_2023_filtrado["reincidente"].value_counts()

df_2023_filtrado:


reincidente
NO    8657
SI    4723
Name: count, dtype: int64

## Descripcion binaria | Es o no anomalía

In [602]:
df_2023_filtrado["Descripcion"].unique()

array(['Solo odt', 'Anomalia verificada y aprobada',
       'Anomalia verificada con acta digital', 'Posible anomalia'],
      dtype=object)

array(['Solo odt', 'Posible anomalia',
       'Anomalia verificada con acta digital',
       'Anomalia verificada y aprobada'], dtype=object)

In [433]:
def crear_columna_anomalia(df):
    """
    Crea la columna 'Anomalia_conf' en un DataFrame basado en la columna 'Descripcion'.

    Parámetros:
    - df (DataFrame): El DataFrame a procesar.
    """

    def determinar_anomalia(descripcion):
        if descripcion == "Solo odt":
            return "No anomalia"
        else:
            return "Anomalia"

    df["Anomalia_conf"] = df["Descripcion"].apply(determinar_anomalia)
    return df

In [606]:
df_2023_filtrado = crear_columna_anomalia(df_2023_filtrado.copy())

df_2023_filtrado.to_csv("df_2023_filtrado_anomalia.csv", index=False)

print("\nDataFrames modificados guardados en archivos CSV.")


DataFrames modificados guardados en archivos CSV.


In [608]:
print(df_2023_filtrado["Anomalia_conf"].value_counts())

Anomalia_conf
No anomalia    21188
Anomalia          66
Name: count, dtype: int64
Anomalia_conf
No anomalia    10987
Anomalia        2393
Name: count, dtype: int64


## Combinar Dataframes

In [610]:
# Crear la columna 'año' en df_2022_filtrado
df_2022_filtrado["año"] = 2022

# Crear la columna 'año' en df_2023_filtrado
df_2023_filtrado["año"] = 2023

# Unir los DataFrames
df_combinado = pd.concat([df_2022_filtrado, df_2023_filtrado], ignore_index=True)

In [612]:
df_combinado["año"].value_counts()

año
2022    21254
2023    13380
Name: count, dtype: int64

In [614]:
print("df_combinado:")
df_combinado.head()

df_combinado:


,item_68,item_248,item_74,item_597,item_602,item_108,item_43,item_603,item_599,item_35,item_598,item_33,item_601,item_24,item_23,item_237,LATI_USU,LONG_USU,NIVEL,AREA,item_288,odt,orden,PLAN_COMERCIAL,Descripcion,reincidente,Anomalia_conf,año
0,False,NO INDICA,False,NOAPL,NOAPL,9.23,636289.0,RA,NOAPL,NO INDICA,DT,64284,DA,SI,SI,True,1.811442,-78.769160,1.0,Urbano,1065193,552285,21286266,31-PLAN RESIDENCIAL TELEMEDIDO,Solo odt,SI,No anomalia,2022
1,False,NO INDICA,False,NOAPL,NOAPL,NO INDICA,NO INDICA,NOAPL,VE,501250.0,VE,501249,DA,NO,NO,True,1.805372,-78.771186,1.0,Urbano,462370,551224,21270629,31-PLAN RESIDENCIAL TELEMEDIDO,Solo odt,SI,No anomalia,2022
2,False,NO INDICA,False,NOAPL,NOAPL,47.5,NO INDICA,NOAPL,VE,656922.0,VE,656921,DA,SI,SI,True,1.805999,-78.767334,1.0,Urbano,903404,552595,21302985,2-PLAN RESIDENCIAL,Solo odt,NO,No anomalia,2022
3,False,NO INDICA,False,NOAPL,NOAPL,8.95,667526.0,RA,VE,661350.0,DT,64420,RA,SI,SI,True,1.796402,-78.793846,1.0,Urbano,8550053,552286,21168129,31-PLAN RESIDENCIAL TELEMEDIDO,Solo odt,NO,No anomalia,2022
4,False,NO INDICA,False,NOAPL,NOAPL,8.5,570158.0,RA,VE,573020.0,VE,573019,DA,SI,SI,True,1.798110,-78.784266,1.0,Urbano,8518138,505002,21063683,2-PLAN RESIDENCIAL,Solo odt,NO,No anomalia,2022


In [618]:
df_combinado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34634 entries, 0 to 34633
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   item_68         34634 non-null  bool   
 1   item_248        34634 non-null  object 
 2   item_74         34634 non-null  bool   
 3   item_597        34634 non-null  object 
 4   item_602        34634 non-null  object 
 5   item_108        34634 non-null  object 
 6   item_43         34634 non-null  object 
 7   item_603        34634 non-null  object 
 8   item_599        34634 non-null  object 
 9   item_35         34634 non-null  object 
 10  item_598        34634 non-null  object 
 11  item_33         34634 non-null  object 
 12  item_601        34634 non-null  object 
 13  item_24         34634 non-null  object 
 14  item_23         34634 non-null  object 
 15  item_237        34634 non-null  bool   
 16  LATI_USU        34634 non-null  float64
 17  LONG_USU        34634 non-null 

In [616]:
# Guardar el DataFrame combinado en un archivo CSV
df_combinado.to_csv("df_combinado.csv", index=False)

print("\nDataFrame combinado guardado en df_combinado.csv.")


DataFrame combinado guardado en df_combinado.csv.
